In [1]:
import os

In [2]:
import random
import tensorflow as tf
import numpy as np

In [3]:
#conversion dictionary
symbol = {
    '+':1,
    '-':-1,
    '*':2,
    '/':-2,
    '=':5
}

symbol_s = {
    1:'+',
    -1:'-',
    2:'*',
    -2:'/',
    5:'='
}

In [81]:
upper_bound=10
lower_bound=1

def generateSets():
    sets = []
    marks=[]
    sol = []
    #ax+b = c
    a = random.randint(lower_bound, upper_bound)
    b = random.randint(lower_bound, upper_bound)
    c = random.randint(lower_bound, upper_bound)
    n = random.randint(0, 1)*2-1
    _b = eval(symbol_s[n]+str(b))
    while c==_b:
        c = random.randint(lower_bound, upper_bound)
    
    x = (c-b)/a
    e = symbol['=']
    
    sol = [
        [a,x,n,b,e,c],
        [a,x,e,c-_b,0,0],
        [x,e,(c-_b)/a,0,0,0]
    ]
    sets.append(sol)
    marks+=[1,1]
    
    sol = [
        [a,x,n,b,e,c],
        [a,x,e,c-_b,0,0],
        [x,e,(c-_b)*a,0,0,0]
    ]
    
    sets.append(sol)
    marks+=[1,0]
    
    sol = [
        [a,x,n,b,e,c],
        [a,x,e,c+_b,0,0],
        [x,e,(c+_b)/a,0,0,0]
    ]
    
    sets.append(sol)
    marks+=[0,1]
    
    return (sets,marks)

In [82]:
#TEST-ONLY
(sets,marks) = generateSets()

In [83]:
training_size = 1000

traindata=[]
trainlabel=[]

#generate training set
#( [step => step],mark )   as 1 training case 
for i in range(training_size):
    (sets, marks) = generateSets()
    for sol in sets:
        #3 sols
        for i in range(2):
            #2steps
            traindata.append(sol[i:i+2])
            
    trainlabel+=marks

In [84]:
#TEST-ONLY
print(len(traindata),len(trainlabel))

6000 6000


In [85]:
#employ model with 6*2 neurons for accuracy improvement

model = tf.keras.models.Sequential([tf.keras.layers.Flatten(), 
                                    tf.keras.layers.Dense(12, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(2, activation=tf.nn.softmax)])

model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss = 'sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [86]:
model.fit(traindata, trainlabel, epochs=10)

Epoch 1/10
188/188 [==============================] - 0s 2ms/step - loss: 0.8599 - accuracy: 0.6278
Epoch 2/10
188/188 [==============================] - 0s 2ms/step - loss: 0.4948 - accuracy: 0.7258
Epoch 3/10
188/188 [==============================] - 0s 2ms/step - loss: 0.4282 - accuracy: 0.7612
Epoch 4/10
188/188 [==============================] - 1s 4ms/step - loss: 0.3881 - accuracy: 0.7893
Epoch 5/10
188/188 [==============================] - 0s 2ms/step - loss: 0.3460 - accuracy: 0.8252
Epoch 6/10
188/188 [==============================] - 0s 2ms/step - loss: 0.3056 - accuracy: 0.8575
Epoch 7/10
188/188 [==============================] - 0s 2ms/step - loss: 0.2669 - accuracy: 0.8933
Epoch 8/10
188/188 [==============================] - 0s 3ms/step - loss: 0.2271 - accuracy: 0.9323
Epoch 9/10
188/188 [==============================] - 1s 3ms/step - loss: 0.1907 - accuracy: 0.9498
Epoch 10/10
188/188 [==============================] - 1s 3ms/step - loss: 0.1615 - accuracy: 0.9587

In [87]:
test_size = 1000

testdata=[]
testlabel=[]

#generate training set
#( [step => step],mark )   as 1 training case 
for i in range(test_size):
    (sets, marks) = generateSets()
    for sol in sets:
        #3 sols
        for i in range(2):
            #2steps
            testdata.append(sol[i:i+2])
            
    testlabel+=marks

In [88]:
model.evaluate(testdata, testlabel)

188/188 [==============================] - 0s 1ms/step - loss: 0.1517 - accuracy: 0.9622


[0.15171509981155396, 0.9621666669845581]

In [89]:
# Save the whole model
model.save('saved_model/model2') 

INFO:tensorflow:Assets written to: saved_model/model2\assets


In [90]:
############  WEB-DEMO BEGINS HERE ############

In [91]:
new_model = tf.keras.models.load_model('saved_model/model2')

In [94]:
#tokenize solution convert:
#empty ''  to    0
#numbers   to    float
#signs     to    token values

def Token(a,ans):
    _a = []
    for step in a:
        _s=[]
        i = 0
        j = 1
        while i < len(step) and j <= len(step):
            #print(step[i:j],i,j)
            if step[i:j] == 'x':
                _s.append(ans)
                i+=1
                j+=1
            elif step[i:j] in symbol.keys():
                _s.append(symbol[step[i:j]])
                i+=1
                j+=1
            else:
                while step[i:j].isnumeric() and j <= len(step):
                    j+=1
                num = int(''.join(step[i:j-1]))
                _s.append(num)
                i=j-1
        _s+=[0]*(6-len(_s))
        _a.append(_s)
    #print(_a)
    return _a

In [102]:
#Practice
ans=2
#formula = "2x+3=7 \n 2x=4 \n x=2"
#formula = "2x+3=7 \n 2x=10 \n x=5"
#formula = "2x+3=7 \n 2x=4 \n x=100"

#adjust formula
sol = formula.split("\n")
for i in range(len(sol)):
    sol[i]=sol[i].strip()

#print question
print(sol[0])

for j in range(1,3):
    #use AI to process
    input_sol = Token(sol[j-1:j+1],ans)
    #print(input_sol)
    result = new_model.predict([input_sol])
    
    #print result
    print(sol[j],end="\t\t")
    print(np.argmax(result),round(max(result[0])*100,2))

2x+3=7
2x=4		1 93.82
x=100		0 100.0


In [ ]:
import tensorflowjs as tfjs

In [ ]:
tfjs.converters.save_keras_model(new_model, "saved_model/modeljs")